# RAG Pipeline Exercise

In this exercise you will build and **compare two simple Retrieval-Augmented Generation (RAG) pipelines**.

You will work with a small collection of PDF documents (e.g. medical guidelines) and:

1. Load and chunk the PDF documents.
2. Create a vector index using **embedding model A** (local `BAAI/bge-m3`).
3. Create a second index using **embedding model B** (e.g. OpenAI or Gemini embeddings).
4. Implement a simple **retriever** and an **answering function** that calls an LLM with retrieved context.
5. Automatically **generate questions** from the documents and use them to **compare two RAG configurations**.

Cells marked with `# TODO` are **for students to implement**.
Everything else is provided scaffolding.

## 0. Setup & Imports

In [25]:
# TODO (easy): skim the imports and make sure you understand what each library is used for.

from dotenv import load_dotenv  # Load environment variables from .env files
import os  # Operating system interface (paths, env vars)
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import glob  # File pattern matching (finding files)
from PyPDF2 import PdfReader  # Reading PDF files
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Splitting text into chunks for RAG
from sentence_transformers import SentenceTransformer  # Generating text embeddings
import faiss  # Efficient similarity search and clustering of dense vectors
import pickle  # Serializing and deserializing Python objects
import random  # Generating random numbers
import numpy as np  # Numerical computing and array operations
import pandas as pd  # Data manipulation and analysis

# LLM / API clients (we will mainly use OpenAI here; Gemini can be added as a bonus)
from openai import OpenAI  # OpenAI API client


In [26]:
# Load API keys from .env (you need to create this file once and add your keys)
load_dotenv()

deepinfra_key = os.getenv("DEEPINFRA_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# For this exercise we mainly use OpenAI for both embeddings (RAG B) and chat completions.
assert openai_api_key is not None, "Please set OPENAI_API_KEY in your .env file."
openai_client = OpenAI(api_key=openai_api_key)


In [27]:
# Make pandas show the full table and full cell content
pd.set_option("display.max_rows", None)       # show all rows
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.max_colwidth", None)   # don't truncate cell text

## 1. Load PDF documents

We assume there is a `data/` folder containing one or more PDF files.

**Task:** implement `load_pdfs(glob_path)` so that it:
- Iterates over all PDF files matching `glob_path`
- Reads them with `PdfReader`
- Concatenates the text of all pages into **one long string**.

In [28]:
def load_pdfs(glob_path: str = "data/*.pdf") -> str:
    """Load all PDFs matching the pattern and return their combined text.

    TODO:
    - Use `glob.glob(glob_path)` to iterate over file paths
    - For each file, open it in binary mode and create a `PdfReader`
    - Loop over `reader.pages` and extract text with the extract_text() function
    - Concatenate everything into a single string `text`
    - Be robust: skip pages where `extract_text()` returns None
    """
    # YOUR CODE HERE
    text = ""
    for pdf_path in glob.glob(glob_path):
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += " " + page_text
    return text



In [29]:
# Run once and inspect
raw_text = load_pdfs("data/*.pdf")
print("Number of characters:", len(raw_text))
print("Preview:", raw_text[:500])

Number of characters: 230708
Preview:  Asthma: diagnosis, 
moni toring and chr onic 
asthma manag emen t (BTS, 
NICE, SI GN) 
NICE guideline 
Published: 27 No vember 202 4 
www .nice.or g.uk/guidance/ng2 45 
© NICE 202 4. All right s reserved. Subject t o Notice of right s (https://www .nice.or g.uk/t erms-and-
conditions#notice-of -right s). Your r esponsi bility 
The r ecommendations in t his guideline r epresent t he view of NICE, arriv ed at aft er car eful 
consideration of t he evidence a vailable. When e xercising t heir judg


## 2. Chunk the text

We will split the long text into overlapping chunks.

Later you can **experiment** with different `chunk_size` and `chunk_overlap` to see how it affects retrieval.

**Task:** start with the given parameters, run once, then try at least one alternative configuration and note the effects.

In [30]:
# Base configuration (RAG A)
chunk_size_a = 2000  # Bigger chunks: captures more context and narrative flow, fewer total chunks
chunk_overlap_a = 200  # Overlap: ensures continuity across boundaries so context isn't lost

splitter_a = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_a,
    chunk_overlap=chunk_overlap_a
)

chunks_a = splitter_a.split_text(raw_text)
print(f"RAG A: {len(chunks_a)} chunks produced, first chunk length = {len(chunks_a[0])}")

# TODO (mini-experiment): change chunk_size / chunk_overlap for RAG B and compare
chunk_size_b = 1000   # Smaller chunks: more granular/precise retrieval, more chunks, but might miss broader context
chunk_overlap_b = 100

splitter_b = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_b,
    chunk_overlap=chunk_overlap_b
)

chunks_b = splitter_b.split_text(raw_text)
print(f"RAG B: {len(chunks_b)} chunks produced, first chunk length = {len(chunks_b[0])}")


RAG A: 130 chunks produced, first chunk length = 1995
RAG B: 260 chunks produced, first chunk length = 979


## 3. Create embeddings and a FAISS index

We start with **Embedding model A: `BAAI/bge-small-en`** using `sentence-transformers`.

Then, as an optional extension, you can build **Embedding model B** using OpenAI or Gemini and compare.

To keep the exercise manageable, the base version only **requires** BGE.

In [31]:
# Embedding model A (local)
model_name_a = "BAAI/bge-small-en"  # Specify the pre-trained model (small and efficient for English)
embedder_a = SentenceTransformer(model_name_a)  # Load the model into memory

# Compute embeddings for all chunks of configuration A
# This converts text chunks into numerical vectors (embeddings)
embeddings_a = embedder_a.encode(chunks_a, convert_to_numpy=True)

dimensions_a = embeddings_a.shape[1]  # Get vector size (e.g., 384 dimensions) required for FAISS
print("Embedding dimensionality (A):", dimensions_a)

index_a = faiss.IndexFlatL2(dimensions_a)  # Create a FAISS index using L2 (Euclidean) distance
index_a.add(embeddings_a)  # Add the calculated embeddings to the index for searching
print("FAISS index (A) size:", index_a.ntotal)

# Persist index/chunks if you like (optional)
os.makedirs("faiss", exist_ok=True)  # Create directory if it doesn't exist
faiss.write_index(index_a, "faiss/faiss_index_a.index")  # Save the vector index to disk
with open("faiss/chunks_a.pkl", "wb") as f:
    pickle.dump(chunks_a, f)  # Save the original text chunks (FAISS only stores vectors, not text)


Embedding dimensionality (A): 384
FAISS index (A) size: 130


In [32]:
# Embedding model B using OpenAI embeddings.

# TODO:
# - Use `openai_client.embeddings.create(...)` to compute embeddings for `chunks_b`
# - Create a second FAISS index `index_b`
# - Make sure to check the dimensionality from the first embedding vector

# Example sketch (not complete, adapt & run if you have API access):
# Initialize OpenAI client
openai_client = OpenAI(api_key=openai_api_key)  # Sets up the client to communicate with OpenAI API

# TODO fulfilled: Use `openai_client.embeddings.create(...)` to compute embeddings
response = openai_client.embeddings.create(
     model="text-embedding-3-small",  # Specifies the model (1536 dimensions for this specific model)
    input=chunks_b  # Sends the list of text chunks to be converted into vectors
)

# Extract the actual embedding vectors from the API response object into a numpy array
embeddings_b = np.array([item.embedding for item in response.data])

# TODO fulfilled: Check dimensionality (crucial because FAISS needs the exact vector size)
dim_b = embeddings_b.shape[1]  # dynamically gets the dimension (e.g., 1536) from the first vector

# TODO fulfilled: Create a second FAISS index `index_b`
index_b = faiss.IndexFlatL2(dim_b)  # Initializes a new index structure for the OpenAI embeddings

index_b.add(embeddings_b)  # Populates the index with the vectors so they can be searched
print("FAISS index (B) size:", index_b.ntotal)


FAISS index (B) size: 260


## 4. Implement a simple retriever

We now implement a generic retrieval function that:
1. Embeds the query.
2. Searches the FAISS index.
3. Returns the corresponding text chunks.

We implement it for configuration A. If you built configuration B, you can reuse the same function.

In [33]:
def retrieve_texts(query: str, k: int, index, chunks, embedder) -> list:
    """Return the top-k most similar chunks for a query.

    TODO (students):
    - Encode the query with `embedder.encode(...)`
    - Call `index.search(query_embedding, k)`
    - Use the returned indices to select the chunks
    - Return a list of strings (chunks)
    """
    # YOUR CODE HERE
    # TODO fulfilled: Encode the query. Wrapped in [query] because the model expects a batch; converted to numpy for FAISS.
    query_emb = embedder.encode([query], convert_to_numpy=True)
    
    # TODO fulfilled: Search the index. Finds the top 'k' closest vectors; returns distances and their integer indices.
    distances, indices = index.search(query_emb, k)
    
    # TODO fulfilled: Use returned indices to select chunks. indices[0] contains the results for our single query.
    retrieved = [chunks[i] for i in indices[0]]
    
    # TODO fulfilled: Return the list of strings (chunks).
    return retrieved

# Quick sanity check
test_query = "What is the most important factor in diagnosing asthma?"
retrieved_text = retrieve_texts(test_query, k=3, index=index_a, chunks=chunks_a, embedder=embedder_a)
print("Number of retrieved chunks:", len(retrieved_text))
print("Preview of first chunk:", retrieved_text[0][:400])


Number of retrieved chunks: 3
Preview of first chunk: and signs of ot her causes of r espirat ory sympt oms but be awar e that e ven if 
examination r esult s are normal, t he person ma y still ha ve ast hma. [NICE 2017] 
Initial tr eatmen t and obje ctive tests f or acu te sym ptoms a t 
presen tation 
1.1.5 Treat people immediat ely if t hey are acut ely unw ell or highly sympt omatic at 
presentation, and per form objectiv e tests that ma y help s


In [34]:
def openai_embed_query(query: str) -> np.ndarray:
    """
    Compute an OpenAI embedding for a single query string and
    return it as a NumPy array of shape (1, dim).
    """
    resp = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=[query]  # list with one string
    )
    vec = np.array(resp.data[0].embedding, dtype="float32")
    return vec.reshape(1, -1)

In [35]:
def retrieve_texts_b(query: str, k: int, index, chunks) -> list:
    """
    Retrieve top-k chunks using OpenAI embeddings for the query.
    """
    query_emb = openai_embed_query(query)  # shape (1, dim)
    distances, indices = index.search(query_emb, k)
    retrieved = [chunks[i] for i in indices[0]]
    return retrieved

## 5. Implement `answer_query` using an LLM

Now we build the actual RAG call:

1. Use `retrieve_texts` to get top-`k` chunks.
2. Concatenate them into a context string.
3. Build a prompt that:
   - shows the context
   - asks the model to answer the user question based **only** on this context.
4. Call the OpenAI chat completion API.

This is the **core RAG function**.

In [36]:
def answer_query_a(query: str, k: int, index, chunks, embedder, client: OpenAI) -> str:
    """RAG-style answer: retrieve context and ask an LLM.

    TODO (students):
    - Use `retrieve_texts` to get `k` relevant chunks.
    - Join them into a single context string.
    - Build a chat prompt that instructs the model to answer *only* using the context.
    - Call `client.chat.completions.create(...)` with model `"gpt-4o-mini"` (or similar).
    - Return the model's answer text.
    """
    # TODO fulfilled: Use `retrieve_texts` to get `k` relevant chunks.
    retrieved_chunks = retrieve_texts(query, k, index, chunks, embedder)
    
    # TODO fulfilled: Join them into a single context string. 
    # We use a separator ("\n\n---\n\n") to clearly distinguish between different source chunks.
    context = "\n\n---\n\n".join(retrieved_chunks)

    # TODO fulfilled: Build a chat prompt that instructs the model to answer *only* using the context.
    system_prompt = (
        "You are a helpful assistant answering questions based ONLY on the provided context. "
        "If the answer is not in the context, say that you do not know."
    )

    # Structure the conversation history for the API (System instruction + User query with context)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
    ]

    # TODO fulfilled: Call `client.chat.completions.create(...)` with model `"gpt-4o-mini"`.
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # TODO fulfilled: Return the model's answer text.
    # We access choices[0] because we only requested one completion (n=1 by default).
    return completion.choices[0].message.content.strip()

# Quick manual test
answer = answer_query_a(test_query, k=3, index=index_a, chunks=chunks_a, embedder=embedder_a, client=openai_client)
print("RAG answer:", answer)


RAG answer: The most important factor in diagnosing asthma is the history suggesting asthma combined with objective tests that may support a diagnosis, such as eosinophil count, fractional exhaled nitric oxide (FeNO), spirometry, and peak expiratory flow (PEF).


### Answer Function with the other embedding type

In [37]:
def answer_query_b(query: str, k: int, index, chunks, client: OpenAI) -> str:
    """
    RAG-style answer for configuration B (OpenAI embeddings + chunking B).
    """
    # Retrieve relevant chunks using the OpenAI-specific retrieval function
    # We pass 'client' here so the function can call the OpenAI embedding API
    retrieved_chunks = retrieve_texts_b(query, k, index, chunks)
    
    # Join the retrieved chunks into a single context string
    # The separator "\n\n---\n\n" helps the LLM distinguish between different text sources
    context = "\n\n---\n\n".join(retrieved_chunks)

    # Define the system prompt to enforce strict context usage (Grounding)
    system_prompt = (
        "You are a helpful assistant answering questions based ONLY on the provided context. "
        "If the answer is not in the context, say that you do not know."
    )

    # Construct the message history: System instructions + User query with injected context
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"},
    ]

    # Call the LLM (GPT-4o-mini) to generate the final answer
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )

    # Extract and return the text content from the model's response
    return completion.choices[0].message.content.strip()

## 6. Generate questions from random chunks (automatic evaluation set)

To compare two RAG configurations, we need **questions**.

We will:
- randomly sample a few chunks from the corpus,
- ask an LLM to generate a **good question** whose answer is contained in the chunk.

Then we can use these question–chunk pairs as a small evaluation set.

We provide most of the implementation. Your job is mainly to:
- inspect the code,
- understand the prompt,
- maybe tweak the number of chunks or retries.

In [38]:
def generate_questions_for_random_chunks(chunks, num_chunks: int = 5, max_retries: int = 2):
    # TODO (tweak): Change 'num_chunks' here if you want a larger evaluation set (e.g., 10 or 20)
    selected_chunks = random.sample(chunks, num_chunks) # Randomly select a subset of chunks to test on
    qa_pairs = []

    for chunk in selected_chunks:
        # TODO (inspect): This prompt instructs the LLM to create a question based on the text.
        # You could modify this to ask for "hard" questions, "true/false" questions, etc.
        prompt = (
            "Based on the following text, generate an insightful question that covers its key content:\n\n"
            "Text:\n" + chunk + "\n\n"
            "Question:"
        )

        question = None
        # Retry loop: handles occasional API errors or timeouts
        for attempt in range(max_retries):
            try:
                # Ask the LLM to generate the question
                completion = openai_client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": prompt}]
                )
                question = completion.choices[0].message.content.strip()
                if question:
                    break # Successfully generated a question, stop retrying
            except Exception as e:
                print("Error while generating question, retrying...", e)

        if question is None:
            question = "Error: could not generate question."

        # Save the pair: (The original text chunk, The generated question)
        # This creates our "Ground Truth" for evaluation later
        qa_pairs.append((chunk, question))

    return qa_pairs

# Execute the function
# TODO: You can increase num_chunks here to generate a bigger test set
questions = generate_questions_for_random_chunks(chunks_a, num_chunks=5, max_retries=2)

for i, (chunk, q) in enumerate(questions, 1):
    print(f"Q{i}: {q}\n  From chunk preview: {chunk[:120]}...\n")


Q1: What factors influenced the committee's decision to retain and update the step 4 treatment recommendations for hypertension, despite a lack of new evidence?
  From chunk preview: developed t o suppor t healt hcare professionals and people wit h hyper tension t o discuss 
their tr eatment options an...

Q2: What are the recommended strategies for monitoring asthma control during patient reviews, according to the guidelines from NICE and BTS/SIGN?
  From chunk preview: specialist. [NICE 2017] 
1.5 M onitoring asthma c ontrol 
1.5.1 Monit or asthma contr ol at every review . In addition t...

Q3: What steps should be taken if a child under 5 with suspected asthma continues to experience symptoms despite a trial of low-dose inhaled corticosteroids and short-acting beta-2 agonists?
  From chunk preview: childr en under 5 . 
1.9.1 Consider an 8 t o12 w eek trial of twice-daily paediatric lo w-dose inhaled 
corticost eroid ...

Q4: What evidence led the committee to recommend low-dose inh

## 7. Compare two RAG configurations

Now we can:
- Use the generated questions,
- Answer them with RAG configuration A (BGE + chunking A),
- (Optional) Answer them with RAG configuration B (e.g. different chunking and/or different embeddings),
- Compare the answers qualitatively.

To keep the exercise manageable, we start with config A only.
If you implemented config B, reuse `answer_query` with `index_b`, `chunks_b`, and your second embedder.

In [39]:
def answer_generated_questions_a(question_tuples, k, index, chunks, embedder, client):
    """
    Runs the RAG pipeline (Config A) on a list of (chunk, question) pairs to evaluate performance.
    """
    results = []
    # Iterate through the evaluation set (ground truth chunk + generated question) created in Step 6
    for chunk, question in question_tuples:
        # Generate an answer using RAG Configuration A
        answer = answer_query_a(question, k, index, chunks, embedder, client)
        
        # Store the triplet: Source Text (Truth), Question, and RAG Answer for comparison
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer
        })
    return results

# Execute the evaluation for Configuration A
results_a = answer_generated_questions_a(
    questions,          # The list of generated questions from Step 6
    k=5,                # Retrieve top 5 chunks for context
    index=index_a,      # Use FAISS index A
    chunks=chunks_a,    # Use Text Chunks A
    embedder=embedder_a,# Use Local Embedder (BGE)
    client=openai_client,
)

# Print the results for manual qualitative inspection
for item in results_a:
    print("Question:", item["question"])
    print("Answer A:", item["answer"])
    # Show a snippet of the original chunk to see if the answer aligns with the source truth
    print("Source chunk preview:", item["chunk"][:150], "...")
    print("-" * 60)


Question: What factors influenced the committee's decision to retain and update the step 4 treatment recommendations for hypertension, despite a lack of new evidence?
Answer A: The committee's decision to retain and update the step 4 treatment recommendations for hypertension was influenced by their significant clinical experience with the safe and effective use of alpha- or beta-blockers. They agreed that adding another drug is likely to have a greater effect on blood pressure than increasing the thiazide diuretic dose, which was supported by their discussion surrounding the lack of evidence for high-dose thiazide diuretics improving cardiovascular outcomes. Additionally, they considered the need for clarity regarding the features that warrant same-day referral and the importance of quickly investigating target organ damage before treatment decisions for those with severely raised blood pressure but no other concerning symptoms.
Source chunk preview: developed t o suppor t healt hcare

### Extension: add RAG B and create a comparison table

If you implemented a second configuration (e.g. different chunking + OpenAI embeddings):

1. Build `index_b` using OpenAI embeddings and `chunks_b`.
2. Implement `openai_embed_query`, `retrieve_texts_b`, and `answer_query_b`.
3. Run `results_b = answer_generated_questions_b(questions, k=5, index=index_b, chunks=chunks_b, client=openai_client)`.
4. For each question, compare:
   - Which answer is more complete / specific?
   - Which one is better grounded in the source chunk?
5. Summarise your findings in a short **markdown cell** or a small table.

---

This concludes the core RAG exercise.


In [40]:
def answer_generated_questions_b(question_tuples, k, index, chunks, client):
    """
    Use RAG B to answer a list of (chunk, question) pairs.
    """
    results = []
    # Loop through the validation set (ground truth chunk + question)
    for chunk, question in question_tuples:
        # Generate answer using the RAG B pipeline (OpenAI embeddings)
        answer = answer_query_b(question, k, index, chunks, client)
        results.append({
            "chunk": chunk,
            "question": question,
            "answer": answer,
        })
    return results


In [41]:
# Run the evaluation for Configuration B
results_b = answer_generated_questions_b(
    questions,          # Use the exact same questions as RAG A for fair comparison
    k=5,                # Number of context chunks to retrieve
    index=index_b,      # The FAISS index built with OpenAI embeddings
    chunks=chunks_b,    # The text chunks corresponding to Config B (e.g., smaller chunks)
    client=openai_client
)


In [42]:
rows = []

# zip() pairs up the results from A and B so we can compare the answers to the same question
for qa_a, qa_b in zip(results_a, results_b):
    rows.append({
        "Question": qa_a["question"],
        "Answer A (BGE + config A)": qa_a["answer"],       # Output from the local model pipeline
        "Answer B (OpenAI + config B)": qa_b["answer"],    # Output from the OpenAI pipeline
        "Source chunk (A) preview": qa_a["chunk"][:200] + "..." # Snippet of the text the question was generated from
    })

# Create a DataFrame to visualize the comparison as a neat table
df_comparison = pd.DataFrame(rows)
display(df_comparison) # Renders the dataframe in the notebook


,Question,Answer A (BGE + config A),Answer B (OpenAI + config B),Source chunk (A) preview
0,"What factors influenced the committee's decision to retain and update the step 4 treatment recommendations for hypertension, despite a lack of new evidence?","The committee's decision to retain and update the step 4 treatment recommendations for hypertension was influenced by their significant clinical experience with the safe and effective use of alpha- or beta-blockers. They agreed that adding another drug is likely to have a greater effect on blood pressure than increasing the thiazide diuretic dose, which was supported by their discussion surrounding the lack of evidence for high-dose thiazide diuretics improving cardiovascular outcomes. Additionally, they considered the need for clarity regarding the features that warrant same-day referral and the importance of quickly investigating target organ damage before treatment decisions for those with severely raised blood pressure but no other concerning symptoms.","The committee's decision to retain and update the step 4 treatment recommendations for hypertension was influenced by several factors:\n\n1. **Clinical Experience**: The committee discussed their clinical experience and deemed it important in the absence of new evidence. \n\n2. **Previous Recommendations**: They considered the previous recommendations from the 2011 guideline, which were based on robust clinical and cost-effectiveness evidence.\n\n3. **Common Clinical Practice**: The use of aldosterone antagonists has become common in clinical practice, warranting the retention of related recommendations despite the lack of robust new evidence.\n\n4. **Ineffectiveness of Higher Doses**: The committee noted that studies had shown no improvement in cardiovascular outcomes at higher doses of thiazide diuretics in patients without resistant hypertension, influencing their perspective on step 4 treatment.\n\n5. **Clinical Safety**: The committee found significant clinical experience supporting the safe and effective use of alpha- or beta-blockers, reinforcing the decision to retain related recommendations.\n\nOverall, the combination of established clinical practice, expert consensus, and safety considerations contributed to the decision to retain and update the recommendations for step 4 treatment.",developed t o suppor t healt hcare professionals and people wit h hyper tension t o discuss \ntheir tr eatment options and mak e informed decisions. \nHow the r ecommenda tions mig ht affect practice \nT...
1,"What are the recommended strategies for monitoring asthma control during patient reviews, according to the guidelines from NICE and BTS/SIGN?","The guidelines do not specify the recommended strategies for monitoring asthma control during patient reviews. Therefore, I do not know the answer to your question.","The recommended strategies for monitoring asthma control during patient reviews according to the guidelines from NICE and BTS/SIGN include:\n\n1. Use of a validated symptom questionnaire (e.g., the Asthma Control Questionnaire, the Asthma Control Test, or the Childhood Asthma Control Test) at any asthma review.\n2. Monitoring the amount of reliever inhaler used, including a check of the prescription record.\n3. Keeping track of the number of courses of oral corticosteroids.\n4. Documenting any admissions to hospital or attendance at an emergency department due to asthma.\n5. Considering fractional exhaled nitric oxide (FeNO) monitoring for adults with asthma at their regular review. \n\nAdditionally, if asthma control is deemed suboptimal, the recommendations suggest referring to the principles of pharmacological treatment.","specialist. [NICE 2017] \n1.5 M onitoring asthma c ontrol \n1.5.1 Monit or asthma contr ol at every review . In addition t o asking about sympt oms, \ncheck: \n• time off w ork or school due t o ast hma \n..."
2,What steps should be taken if a child under 5 with suspected asthma continues to experience 